## This notebook is used for testing MTAOS communication with the AOS components: M1M3, M2, and hexapods

Step-by-step description of what we do can be found in 
Jira test case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T???? 

test on NTS first:

* Do all the callback messages look right?
* Look at the EFD notebooks at the same time. Is everything as expected?

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import yaml

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from aosTools import *

plt.jet();

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=7)

In [ ]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
mtcs.components_attr

In [ ]:
camhex = mtcs.rem.mthexapod_1
m2hex = mtcs.rem.mthexapod_2
m1m3 = mtcs.rem.mtm1m3
m2 = mtcs.rem.mtm2
aos = mtcs.rem.mtaos
mount = mtcs.rem.mtmount
rot = mtcs.rem.mtrotator

In [ ]:
def printLogMessage(data):
    print(f"{data.level}: {data.message}")
aos.evt_logMessage.callback = printLogMessage  #None
await aos.cmd_setLogLevel.set_start(level=10, timeout=5)

In [ ]:
await aos.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.ENABLED) #, settingsToApply='default') #leave this out!!
#two levels of defaults: csc defaults, config repo; right now csc defaults, they will move to config repo; 
#but to load defaults from config repo, we do not need to supply settingsToApply

In [ ]:
#summit = 1 #use this for summit testing
summit = 0 #use this for NCSA

In [ ]:
from lsst_efd_client import EfdClient

if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1

#### Check summary state of each CSC

In [ ]:
await checkAOSSummaryStates(aos, m1m3, m2, camhex, m2hex)

### Get M1M3 Ready: raise mirorr, turn on FB, clear forces

In [ ]:
await m1m3.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
print('Re-enabling M1M3')
#await salobj.set_summary_state(m1m3, salobj.State.STANDBY) 
await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM1M3(m1m3)

### Get M2 Ready: turn on FB, clear forces

In [ ]:
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(m2, salobj.State.ENABLED) 
#await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM2(m2)

### Get cam hex Ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

In [ ]:
await camhex.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await mount.cmd_moveToTarget.set_start(elevation=75, azimuth=0)

In [ ]:
await rot.cmd_move.set_start(position=0)

In [ ]:
#await camhex.cmd_move.set_start(x=0,y=0,z=100, u=0,v=0,w=0,sync=True)

In [ ]:
await readyHexaForAOS(camhex)

In [ ]:
await camhex.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.ENABLED, settingsToApply="default")

In [ ]:
state = await camhex.evt_summaryState.aget(timeout=5)
print('staring with: cam hex state', salobj.State(state.summaryState), pd.to_datetime(state.private_sndStamp, unit='s'))
dstate = await camhex.evt_controllerState.aget(timeout=5)
print('          cam hex state', MTHexapod.EnabledSubstate(dstate.enabledSubstate), 
  pd.to_datetime(dstate.private_sndStamp, unit='s'))

In [ ]:
mount_angle = await mount.tel_elevation.next(flush=False, timeout=10.)
print("mount elevation angle", mount_angle.actualPosition)
elev = mount_angle.actualPosition

### Get M2 hex Ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

In [ ]:
await m2hex.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
#await m2hex.cmd_move.set_start(x=0,y=0,z=100, u=0,v=0,w=0,sync=True)

In [ ]:
await readyHexaForAOS(m2hex)

In [ ]:
await salobj.set_summary_state(m2hex, salobj.State.ENABLED, settingsToApply="default")

In [ ]:
await aos.cmd_resetCorrection.set_start()
await aos.cmd_issueCorrection.set_start() 

### Step 1: add aberrations via OFC

In [ ]:
zernikes = np.zeros(19)
zerIdx = np.arange(4, 23)
zernikes[7-4] = 1 #add 1um of z7
print(zerIdx)
print(zernikes)

In [ ]:
#x0 makes sure corrections double when same aberrations are added twice
config = {"xref": "x0"}  
config_yaml = yaml.safe_dump(config)

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes, config=config_yaml)

In [ ]:
ofc_dict =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

#### Look at telemetry to verify the changes in the DOFs

In [ ]:
end = Time(datetime.now(), scale='tai')
#end = Time('2021-05-14T18:25:30', scale = 'tai')
start = end - timedelta(seconds=100)

dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedActiveOpticForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.logevent_uncompensatedPosition', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
#plt.plot(dfm.actualPosition, '--', label='mount elevation')
plt.plot(dfm1m3.zForces0*1e3, label='M1M3 aos force 101 (x1000N)')
plt.plot(dfm2.applied0*1e2, label='M2 aos force B1 (x100N)')
plt.plot(dfh1.y, '*', label='Camera hexapod y (microns)')
plt.plot(dfh2.y, 'o', label='M2 hexapod y (microns)')
plt.grid()
plt.legend()

### Step 2: disable M2 hexapod, then try to add same aberrations again. The corrections will be rejected by M2 hexapod.

In [ ]:
#put M2 hexapod in DISABLED state (so that we can test command rejection)
await salobj.set_summary_state(m2hex, salobj.State.DISABLED)

In [ ]:
#rejection only happens when we issue corrections
await aos.cmd_addAberration.set_start(wf = zernikes, config=config_yaml)

In [ ]:
ofc_dict2 =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
#expect to see an error with this command
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict2 = await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot=True)

#### What to expect from the above plot? Everything on the component side is undone, and the corrections are rejected.

Ref:
https://github.com/lsst-ts/ts_MTAOS/blob/a420c25ab868a932c2cbc6fd45fcbb66f0176c65/python/lsst/ts/MTAOS/mtaos.py#L268

For now, let's examine what happened during and after the rejection.

In [ ]:
lastCmd = await aos.evt_rejectedM2HexapodCorrection.aget(timeout=5.)
end = Time(pd.to_datetime(lastCmd.private_sndStamp+0.5, unit='s'), scale= 'tai')
start = end - timedelta(seconds=1)

dfissue = await client.select_time_series('lsst.sal.MTAOS.command_issueCorrection', '*', start, end, csc_index)

dfdof = await client.select_time_series('lsst.sal.MTAOS.logevent_degreeOfFreedom', '*', 
                                        start-timedelta(minutes=30), end, csc_index) #this is from earlier
dfreject = await client.select_time_series('lsst.sal.MTAOS.logevent_rejectedDegreeOfFreedom', '*', start, end, csc_index)

dfC = await client.select_time_series('lsst.sal.MTAOS.logevent_m2HexapodCorrection', '*', start, end, csc_index)
dfCm2 = await client.select_time_series('lsst.sal.MTAOS.logevent_m2Correction', '*', start, end, csc_index)
dfrejectC = await client.select_time_series('lsst.sal.MTAOS.logevent_rejectedM2HexapodCorrection', '*', start, end, csc_index)

cmdm1m3 = await client.select_time_series('lsst.sal.MTM1M3.command_applyActiveOpticForces', '*', start, end, csc_index)
cmdm2 = await client.select_time_series('lsst.sal.MTM2.command_applyForces', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.command_move', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
cmdh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
cmdh2 = dfh[idx2]

dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedActiveOpticForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', 
                                       start-timedelta(seconds=0.5), end+timedelta(seconds=0.5), csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.logevent_uncompensatedPosition', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

In [ ]:
#check that the rejected DOF matches the latest DOF event
a1=np.array([dfdof['aggregatedDoF%d'%i][-1] for i in range(50)])
v1=np.array([dfdof['visitDoF%d'%i][-1] for i in range(50)])
a2=np.array([dfreject['aggregatedDoF%d'%i][-1] for i in range(50)])
v2=np.array([dfreject['visitDoF%d'%i][-1] for i in range(50)])
plt.plot(a1, label='agg dof')
plt.plot(v1,'-*', label='visit dof')
plt.plot(a2,'o', label='rejected agg')
plt.plot(v2,'x', label='rejected visit')
plt.legend()

In [ ]:
#in M2 simulator, we only publish telemetry at 1Hz; This makes it impossible to see if M2 forces changed or not.
dfm2

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot(dfissue.private_host,'v',label='issueCorrection')
plt.plot(dfm1m3.zForces0*1e3, label='M1M3 aos force 101 (x1000N)')
plt.plot(dfm2.applied0*1e2, label='M2 aos force B1 (x100N)')
plt.plot(dfh1.y, '-*', label='Camera hexapod y (microns)')
plt.plot(dfh2.y, '-o', label='M2 hexapod y (microns)')
plt.plot(cmdm1m3.zForces0*1e3,'x', label='M1M3 cmd')
plt.plot(cmdm2.axial0*1e2,'v', label='M2 cmd')
plt.plot(cmdh1.y,'*', label='camhex cmd')
plt.plot(cmdh2.y,'o', label='M2 hex cmd')
plt.grid()
plt.legend()

camhex `evt_uncompensatedPosition` comes before `cmd_move`? Time synchronization issue? Do we need to care?

No command issued when undoing camera hexapod?

In [ ]:
cmdh1

In [ ]:
#we did not even issue M2 hex command; we checked M2 hex status before we issue the cmd
cmdh2

### Step 3. Fix the issue, and re-issue the corrections

Previous corrections have been rejected using `cmd_rejectCorrection`, as part of the error handling. 
We need to do `cmd_addAberration` again.
(even though there is no updated events such as `aos.evt_m1m3Correction`)

Ref:
https://github.com/lsst-ts/ts_MTAOS/blob/a420c25ab868a932c2cbc6fd45fcbb66f0176c65/python/lsst/ts/MTAOS/mtaos.py#L533

In [ ]:
#fix the issue
await salobj.set_summary_state(m2hex, salobj.State.ENABLED, settingsToApply="default")

In [ ]:
await readyHexaForAOS(m2hex)

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes, config=config_yaml)

In [ ]:
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict3 = await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot=True)

In [ ]:
config_yaml

This depends on the control strategy we are using. For x00, do not expect 2 or 1.

In [ ]:
ofc_dict['visit_dof'][2], ofc_dict3['visit_dof'][2]

In [ ]:
#expect 2
ofc_dict3['camhexC'][:5]/ofc_dict['camhexC'][:5]

In [ ]:
#expect 2
ofc_dict3['m2hexC'][:5]/ofc_dict['m2hexC'][:5]

In [ ]:
#expect 2
plt.plot(ofc_dict3['aggregated_dof']/ofc_dict['aggregated_dof'])

In [ ]:
#expect 1
plt.plot(ofc_dict3['visit_dof']/ofc_dict['visit_dof'])

### Step 4. Reject the last `cmd_addAberration`.

If the corrections were applied successfully, and we want to recalculate the corrections, we need to do `cmd_rejectCorrection` first.
`cmd_rejectCorrection` resets the `aggregatedDOF` back to last visit, then we can recalculate the corrections
If the corrections had been applied without error, this will supersede the corrections on the component side as well.

Ref:
https://github.com/lsst-ts/ts_MTAOS/blob/a420c25ab868a932c2cbc6fd45fcbb66f0176c65/python/lsst/ts/MTAOS/mtaos.py#L300

If we simply want to revert back to the system state from last visit, just do `cmd_issueCorrection` after `cmd_rejectCorrection`.

If after `cmd_rejectCorrection`, we simply want to look at the wavefront measurements, and say, I want to try adding 1um of M1M3 bending #5. How do I do that? This is unlikely, because the better thing to do is to let OFC determine the magnitude of this bending mode, by freezing all the other DOFs. If you really want to specify a hand-picked number, perhaps the best way is to use a MTAOS controller, to publish correction events, then do `cmd_issueCorrection`.
The correction events include DOF events and correction events.
You will need to write a function. You pass it a 50x1 array for DOF. 
It loads OFC libaries, calculates forces etc, and publish all the events that `cmd_issueCorrection` needs.

In [ ]:
await aos.cmd_rejectCorrection.set_start()

#### has `cmd_rejectCorrection` reverted both DOFs and the corrections to be sent? (it is not expected to change anything on the component side)

In [ ]:
#cmd_rejectCorrection does not publish new DOF event. Should it?
ofc_dict4 =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
#expect this to be 1.
plt.plot(ofc_dict4['aggregated_dof']/ofc_dict['aggregated_dof'])

In [ ]:
#expect this to be 1/2.
plt.plot(ofc_dict4['aggregated_dof']/ofc_dict3['aggregated_dof'])

### Step 5. add twice aberration. The resulting `aggregatedDOF` should be 3 times the original.

In [ ]:
zernikes[7-4] = 2 # z7 = 2um

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes, config=config_yaml)

In [ ]:
await aos.cmd_issueCorrection.set_start()

In [ ]:
ofc_dict5 =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
#expect this to be 3.
plt.plot(ofc_dict5['aggregated_dof']/ofc_dict['aggregated_dof'])
plt.ylim([2,4])

### Close up

In [ ]:
#if we started with enabled state, we need to put it back
await salobj.set_summary_state(m1m3, salobj.State.ENABLED,  settingsToApply='Default')

In [ ]:
await salobj.set_summary_state(m2, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(m1m3, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.STANDBY)

In [ ]:
await lowerM1M3(m1m3)

### Check if the telescope is in tracking mode. If yes, need to stop stacking. 
The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await ptg.cmd_stopTracking.set_start(timeout=5.)